In [1]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
import datetime
from tqdm.notebook import tqdm
tqdm.pandas()
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [3]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [4]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [5]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 9
    """
entity_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [6]:
DEATH_QUERY = \
    """
    SELECT
    MORTALITY_STS_CD, DEATH_DT, MED_EDU_NBR, PARTY_ID, INSERT_TMSTP, UPDATE_TMSTP, DEATH_CONF_DT
    FROM
    AMAEDW.PERSON
    WHERE
    ACTIVE_IND = 'Y'
    AND
    MORTALITY_STS_CD is not null
    """
DEATH_EDW = pd.read_sql(con=AMAEDW, sql=DEATH_QUERY)

In [7]:
DEATH_QUERY = \
    """
    SELECT
    ENTITY_ID, MORTALITY_STS, DEATH_DT, UPDATE_DTM, CONF_DT
    FROM
    informix.MORTALITY_INFO_GT
    WHERE
    MORTALITY_STS is not null
    """
DEATH_AIMS = pd.read_sql(con=informix, sql=DEATH_QUERY)

In [17]:
DEATH_AIMS['ENTITY_ID'] = [str(x) for x in DEATH_AIMS.entity_id]

In [18]:
# IDS = pd.merge(entity_ids, party_ids, on='PARTY_ID')
# test = pd.merge(IDS, ov_me, left_on='ME', right_on='medical_education_number')
# test_cont = pd.merge(test, DEATH_EDW, on='PARTY_ID').drop_duplicates()
all_death = pd.merge(test_cont, DEATH_AIMS, on='ENTITY_ID', how='left').drop_duplicates()

In [21]:
hmm = all_death[~all_death.DEATH_DT.isna()]

In [25]:
hmm.dtypes

PARTY_ID                             int64
ENTITY_ID                           object
ME                                  object
medical_education_number            object
type                                object
MORTALITY_STS_CD                    object
DEATH_DT                            object
MED_EDU_NBR                         object
INSERT_TMSTP                datetime64[ns]
UPDATE_TMSTP                datetime64[ns]
DEATH_CONF_DT                       object
entity_id                            int64
mortality_sts                       object
death_dt                            object
update_dtm                  datetime64[ns]
conf_dt                             object
dtype: object

In [29]:
mismatch_list = []
for row in hmm.itertuples():
    mismatch_list.append(row.UPDATE_TMSTP.date() != row.update_dtm.date())

In [30]:
hmm[mismatch_list]

,PARTY_ID,ENTITY_ID,ME,medical_education_number,type,MORTALITY_STS_CD,DEATH_DT,MED_EDU_NBR,INSERT_TMSTP,UPDATE_TMSTP,DEATH_CONF_DT,entity_id,mortality_sts,death_dt,update_dtm,conf_dt
912,1943352,3319858,01611391320,01611391320,Physician,P,2002-01-01,01611391320,2005-12-16 18:28:15,2015-08-14 12:27:27,2014-06-02,3319858,P,2002-01-01,2014-09-04 17:21:49.966,None
1128,1978763,4853533,02307390127,02307390127,Physician,P,2014-07-22,02307390127,2005-12-16 18:28:15,2015-08-14 12:27:27,2014-11-05,4853533,P,2014-07-22,2014-11-05 10:51:35.167,None
1604,2048008,4412261,02012940893,02012940893,Physician,C,2022-02-15,02012940893,2005-12-16 18:28:15,2022-05-16 20:39:35,2022-05-16,4412261,P,None,2022-02-23 14:37:17.432,None
1612,2048403,4171982,01902470324,01902470324,Physician,P,1989-04-19,01902470324,2005-12-16 18:28:15,2015-08-14 12:27:27,2011-10-26,4171982,P,1989-04-19,2011-10-31 01:24:21.208,None
1771,2067465,5527193,02501852301,02501852301,Physician,P,2011-04-01,02501852301,2005-12-16 18:28:15,2015-08-14 12:27:27,2011-10-27,5527193,P,2011-04-01,2012-04-10 10:18:41.611,None
2077,2114349,4442148,02101380761,02101380761,Physician,P,1996-09-17,02101380761,2005-12-16 18:28:15,2015-08-14 12:27:27,2011-10-26,4442148,P,1996-09-17,2011-10-31 01:24:21.208,None
2169,2129730,4857459,02307430498,02307430498,Physician,P,1996-01-01,02307430498,2005-12-16 18:28:15,2015-08-14 12:27:27,2011-11-16,4857459,P,1996-01-01,2011-11-16 13:46:45.387,None
2537,2177607,6537826,03005780887,03005780887,Physician,C,2022-04-10,03005780887,2005-12-16 18:28:15,2022-05-12 20:39:25,2022-05-12,6537826,P,None,2022-04-12 12:20:45.917,None
2583,2184750,6896236,03501541060,03501541060,Physician,P,2012-12-12,03501541060,2005-12-16 18:28:15,2015-08-14 12:27:27,2013-08-05,6896236,P,2012-12-12,2013-08-05 12:16:27.387,None
2707,2198776,7200592,03508571274,03508571274,Physician,P,2012-03-24,03508571274,2005-12-16 18:28:15,2015-08-14 12:27:27,2012-06-28,7200592,P,2012-03-24,2012-06-28 14:54:29.590,None
